[View in Colaboratory](https://colab.research.google.com/github/NTT123/SGLD_AEVI/blob/master/Arithmetic_Coding.ipynb)

#Arithmetic Coding 

Arithmetic coding (AC) là thuật toán nén dữ liệu dựa trên lý thuyết thông tin. Nó cho phép chúng ta lưu trữ thông tin với số lượng bits tối thiểu. Claude  Shannon chỉ ra rằng không thể nào lưu trữ thông tin với số bits nhỏ hơn entropy  của thông tin này. AC cho phép chúng ta tiến tới gần giới hạn entropy này với khoảng cách 2 bits.

## Giới thiệu về nén dữ liệu


Đối tượng ta đang xem xét ở đây là **thông tin** và câu hỏi ta muốn trả lời là: cần bao nhiêu bits dữ liệu để lưu trữ thông tin trên máy tính?

Giả sử rằng thông tin $T$ ta muốn truyền đi hay lưu trữ là một trong 4 khả năng $$ T \in \left\{ a, b, c, d\right\}$$
Một phương pháp đơn giản để lưu trữ thông tin là ra sẽ dùng $\log_2 (4)$ bits thông tin để lưu trữ $T$. Ta xây dựng một bảng dùng để encode và decode thông tin như sau:

| Symbol | Code |
|--------|------|
| a      | 00   |
| b      | 01   |
| c      | 10   |
| d      | 11   |

Như vậy ta cần 2 bits để lưu trữ $T$ trong cả 4 giá trị có thể nhận của nó.

Tuy nhiên, ta nhận thấy cách thức mã hóa như trên chỉ phù hợp khi xác suất $T$ nhận một giá trị bất kì là bằng nhau cho mọi khả năng:
$$
P(T = a)  = P(T= b) = P(T=c) = P(T = d) = \frac 1 4
$$

Do đó, độ dài mà ta kỳ vọng khi lưu trữ $T$ trên máy tính là:
$$
\mathrm E[\textrm{length}(\textrm{code}(T))] = \frac 1 4 \times 2 + \frac 1 4 \times 2 + \frac 1 4 \times 2 + \frac 1 4 \times 2 = 2
$$

Nếu phân phối xác suất của $a, b, c, d$ không đồng đều thì ta có một phương pháp tốt hơn.
Giả dụ, ta có bảng phân phối xác suất như sau:

| Symbol | Probability |
|--------|------|
| a      | 0.9  |
| b      | 0.05   |
| c      | 0.025  |
| d      | 0.025  |

Thuật toán tối ưu được dùng ở đây là Huffman coding (được sử dụng rộng rãi trong định dạng file MP3 và JPEG).

**Huffman coding**:

Bước 0: khởi tạo $code(x) = \emptyset$, cho mọi $x \in \{a, b, c, d\}$

Bước 1: Chọn hai khả năng có xác suất nhỏ nhất.
Ở đây ta chọn $c$ và $d$.

Bước 2: Thêm $1$ và $0$ vào code của hai trường hợp được chọn.

$\textrm {code}(c) \gets \textrm {code}(c) + 1 = 1$ và $\textrm{code} (d)  \gets \textrm {code}(d) + 0 = 0$.

Bước 3: Gộp hai trường hợp được chọn thành một trường hợp mới với xác suất là tổng của hai xác suất nhỏ nhất.
Ở đây ta thay $c$ và $d$ bởi $\{c, d\}$ với xác suất $0.05$.

Quay lại bước 1


Kết thúc quá trình này, ta xây dựng được 1 cây nhị phân như sau:

```
                      .
                     / \
                  1 /   \ 0
                   /      \
                  a        .
                          / \
                     1   /   \  0
                        /     \
                       b       .
                              / \
                          1  /   \  0
                            /     \
                           c       d
                      

```

Bảng encode/decode tương ứng là:

| Symbol | Code |
|--------|------|
| a      | 1   |
| b      | 01   |
| c      | 001   |
| d      | 000   |

Độ dài bits mà ta kỳ vọng khi lưu trữ thông tin của $T$ lúc này là:
$$
\mathrm E[\textrm{length}(\textrm{code}(T))] = 1 \times 0.9 + 2 \times 0.05 + 3 \times 0.025 + 3 \times 0.025 = 1.15~~\mathrm{ bits }
$$



In [251]:
print(1*0.9 + 2* 0.05 + 3 * 0.025*2, " bits")

1.15  bits


Claude  Shannon định nghĩa entropy của $T$ như sau:
$$H(T) = \sum_{x \in \left\{ a, b, c, d\right\}}  \log\left( \frac 1  {P(x)}\right) \times P(x) = \mathrm E\left[ \log\left(\frac {1}{P(T)}\right)\right] $$

Từ đó ông chứng minh rằng, 
$$
\mathrm E[\textrm{length}(\textrm{code}(T))] ]\geq H(T)
$$

Thuật toán Huffman coding đảm bảo rằng:
$$
\mathrm E[\textrm{length}(\textrm{code}(T))] ]\leq H(T) + 1
$$

Ta thử tính entropy của $T$,
$$
H(T) = -0.9 \times \log(0.9) - 0.05 \times \log(0.05) - 2 \times 0.025 \times \log(0.025)  \approx 0.619 ~~\mathrm{bits} 
$$



In [252]:
import math
print( -0.9 * math.log2(0.9) - 0.05 * math.log2(0.05) - 2 * 0.025 * math.log2(0.025), " bits")

0.6189955935892812  bits


## Arithmetic coding

Một giới hạn của Huffman coding là nó chỉ hoạt động khi $T$ nhận giá trị từ một tập hữu hạn các khả năng có thể. Trong  thực tế, ta muốn nén một file có độ dài bất kỳ. Nói cách khác, ta muốn tìm một phương pháp nén mà $T$ có thể nhận vô hạn các khả năng khác nhau.

Thậm chí nếu ta giới hạn $T$ là một chuỗi hữu hạn 512 bits, thì Huffman coding cần xây dựng một cây nhị phân có $2^{512}$ lá. Điều này là không khả thi trong thực tế.

Còn nếu ta áp dụng thuật toán Huffman coding cho mỗi một chuỗi $5$ bits, thì ứng với mỗi 5 bits này, ta có  thể bị phung phí 1 bit so với giới hạn entropy của 5 bits này. Số lượng bits bị phung phí sẽ cộng dồn càng nhiều với một file có độ dài càng lớn.

Thêm vào đó, phân phối xác suất của bit hiện thời rất nhiều khả năng là phụ thuộc vào giá trị của các bits xuất hiện trước nó trong chuỗi bit. Cho nên ta muốn xây dựng một phương pháp nén khai thác việc phụ thuộc này.


Ta sẽ lấy ví dụ cho phần này như sau:

Xét một đồng xu $X$ bị thiên vị:
$$
P(X =  1) = 0.9
$$
và
$$
P(X =0) = 0.1
$$

Ta muốn lưu trữ một chuỗi 512  bits $T = x_1 \dots, x_{512}$ sao cho:
$$
x_i \sim P(X)
$$

Nếu ta dùng Huffman coding, thì ta cần xây dựng 1 cây nhị phân $2^{512}$ lá! Còn nếu ta dùng Huffman coding cho mỗi $x_i$, thì ta cần $512$ bits để lưu trữ vì mỗi $x_i$ cần $1$ bit để mã hóa.

Arithmetic coding giải quyết bài toán trên bằng cách encode mỗi khả năng $x$ của $T$, trong $2^{512}$ khả năng, bằng một khoảng $[a, b] \subset [0, 1]$ sao cho độ dài của khoảng này $b - a$ bằng  chính xác suất của $x$:
$$
\begin{align}
\textrm{code}(x) &= [a, b], \\
\textrm{such that}~~~~~~~~~~~~~~~~~& \\
P(x) &= b- a
\end{align}
$$

Nói cách khác, ta chia khoảng số thực $[0, 1]$ ra thành $2^{512}$ khoảng nhỏ, sao cho độ dài mỗi khoảng là xác suất  $T$ nhận giá trị tương ứng.

Tiếp theo, để xác định được khoảng $[a, b]$ tương ứng với một giá trị $x = x_1, \dots, x_{512}$ bất kì, ta dùng chain rule:
$$
P(x) = P(x_1) \times P(x_2 \mid x_1) \times P(x_3 \mid x_1, x_2) \times \cdots \times P(x_{512} \mid x_1,\dots, x_{511})
$$

Nói cách khác, ta cần một probabilistic model  của $T$ cho phép ta tính $P(x_t \mid x_1, \dots, x_{t-1})$, i.e., auto-regressive model. 

Trong ví dụ của ta, $$P(x_t \mid x_1, \dots, x_{t-1}) = P(X)$$

Bây giờ ta bắt đầu xét khoảng $k^{(1)} = [0, 1]$ và $x_1$, ta sẽ chia ra $k$ ra thành hai khoảng con : $k^{(1)}_1 = [0, 0.9]$ và $k^{(1)}_0 = [0.9, 1.0]$.

Nếu $x_1 = 1$, ta sẽ chọn $k^{(2)} = k^{(1)}_1$ và ngược lại nếu $x_1 = 0$ ta chọn $k^{(2)} = k^{(1)}_0$,
$$
k^{(2)} = k^{(1)}_{x_1}
$$

Giờ ta xét $x_2$, ta cũng chia khoảng $k^{(2)}$ ra làm 2 khoảng con $k^{(2)}_1$ và $k^{(2)}_0$ với tỉ lệ tương ứng là $0.9$ và $0.1$. Tùy thuộc vào giá trị của $x_2$ mà ta chọn được khoảng $k^{(3)}$ phù hợp.

Lặp lại các hành động này $512$ lần cho ta khoảng $[a, b]$ ứng với $x$.

Tiếp theo,  ta sẽ giới thiệu cách để encode khoảng $[a, b]$ bởi chuỗi bits $y= y_1, \dots, y_t$  sao cho $y_i \in \left\{0, 1\right\}$.
Tương tự như cách làm với $x$, tuy nhiên, ở đây $P(y_i = 1) = 0.5$.

Ta chia khoảng $[0, 1]$ ra làm hai khoảng con  $[0, 0.5]$ ứng với $y_1=1$ và $[0.5, 1.0]$ ứng với $y_1 = 0$, ta sẽ chọn giá trị của $y_1$ sao cho khoảng tương ứng của nó chứa $[a, b]$.

Lặp lại quá trình này $t$ lần cho tới khi ta tìm được một khoảng $[t, q]$ nằm trong $[a, b]$,
$$
[t, q] \subset [a, b]
$$

Vì mỗi lần ta giảm độ dài của khoảng đi $\frac 1 2 $, nên ước tính, ta cần tối đa $\log_2 \frac 1 {b-a} + 2= \log_2 \frac 1 {P(x)} + 2 $, bits để encode khoảng $[a, b]$.

Cũng bởi vì vậy mà ta đảm bảo được rằng, độ dài mà ta kì vọng khi encode $T$ không lớn hơn $2$ bits so với entropy của $T$.

## Thực hành

Ta sinh ra một chuỗi dài $512$ bits như bên dưới:

In [253]:
#@title
import numpy as np
from fractions import Fraction


p = Fraction(9, 10)

def Bernoulli(p):
    if np.random.uniform() < p:
        return 1
    else:
        return 0

def tostr(bits):
    txt =  "".join(str(i) for i in bits)
    import textwrap
    return textwrap.fill(txt, 32)

bits = [Bernoulli(p) for i in range(512)]

print(tostr(bits))

01111111111111101101011101111111
10100111101111101111111111111111
10111010111111111111111111101010
11001111111011111111111111111101
11110111111111111111111111111111
01111111110111111111101110111111
11111010011111111111111110110111
11111011101111111111111111111111
11111111111111111110111111111111
11010111111111111111111111111011
11111111111010111111111101111111
11110111111111010111111111111111
11011110111111111111111111111111
11110111111111111111101111111111
11010111111111110111111111111111
01011101111011111111111111111111


Entropy của chuỗi nhị phân $x$ là:

In [254]:
#@title
entropy =  (-math.log2(p) * p - math.log2(1-p) * (1-p)) * 512
print("Entropy: ", entropy, " bits")

Entropy:  240.125743917712  bits


### Encoding

Tiếp theo, ta sẽ encode chuỗi $512$ bits này bởi khoảng số thực $[a, b]$:

In [255]:
def findEncodeSegment(bits):
    left = Fraction(0, 1)
    right = Fraction(1, 1)
    
    for xi in bits:
        boundary = left + (right-left) * (1-p)
        left, right = [ (left, boundary), (boundary, right) ] [xi]
    
    return left, right

In [256]:
seg = findEncodeSegment(bits)
left, right = seg
print("[a, b] = [%0.5f, %0.5f]" % (float(left), float(right)))

[a, b] = [0.07758, 0.07758]


Sau đó ta sẽ encode khoảng $[a, b]$ bởi 1 chuỗi bits nhị phân $y$:

In [257]:
def encodeSegmentInBinary(seg):
    left, right = seg
    mid = (left + right) / 2
    
    l = Fraction(0, 1)
    r = Fraction(1, 1)
    zipbits = []
    while True:
        m = (l + r) / 2
        if m <= mid:
            l, r= m, r
            zipbits.append(1)
        else:
            l, r = l, m
            zipbits.append(0)
        if l > left  and r < right:
            break
    return zipbits, l, r

In [258]:
zipbits, l, r = encodeSegmentInBinary(seg)
print("y = '''\n%s '''\n" % tostr(zipbits))
print("Length: ", len(zipbits), " bits \n")
print(l)
print(r)

y = '''
00010011110111000110001101111111
10001001110000101100000100110010
01110110010010111010101100000011
01100100010011001011000110100100
00100101011001010010110011011000
11000011000111011010110111001001
11111000101011111101111111111011
11101100100000000011011 '''

Length:  247  bits 

17545580446567902951110030117363632688713002338361434669305437523904184347/226156424291633194186662080095093570025917938800079226639565593765455331328
4386395111641975737777507529340908172178250584590358667326359380976046087/56539106072908298546665520023773392506479484700019806659891398441363832832


### Decoding



Tại đây ta sẽ thực hiện các thao tác ngược để giải mã chuỗi bits $y$:

In [259]:
def decodeBinaryToSegment(zipbits):
    l = Fraction(0, 1)
    r = Fraction(1, 1)
    for bit in zipbits:
        m = (l + r) / 2
        l, r = [(l, m), (m, r) ][bit]
    return l, r

In [260]:
decode_seg = decodeBinaryToSegment(zipbits)
print(decode_seg[0])
print(decode_seg[1])

17545580446567902951110030117363632688713002338361434669305437523904184347/226156424291633194186662080095093570025917938800079226639565593765455331328
4386395111641975737777507529340908172178250584590358667326359380976046087/56539106072908298546665520023773392506479484700019806659891398441363832832


In [261]:
def decodeSegmentToBits(decode_seg):
    l, r = decode_seg
    m = (l + r) /2
    left = Fraction(0, 1)
    right = Fraction(1, 1)
    bits = []
    for i in range(512):
        boundary = left + (right-left) * (1-p)
        if boundary  < m:
            bit = 1
        else:
            bit = 0
            
        left, right = [ (left, boundary), (boundary, right) ] [bit]

        bits.append(bit)
    return bits

In [262]:
unzipbits = decodeSegmentToBits(decode_seg)

Chuỗi bits sau decode là:

In [263]:
print(tostr(unzipbits))

01111111111111101101011101111111
10100111101111101111111111111111
10111010111111111111111111101010
11001111111011111111111111111101
11110111111111111111111111111111
01111111110111111111101110111111
11111010011111111111111110110111
11111011101111111111111111111111
11111111111111111110111111111111
11010111111111111111111111111011
11111111111010111111111101111111
11110111111111010111111111111111
11011110111111111111111111111111
11110111111111111111101111111111
11010111111111110111111111111111
01011101111011111111111111111111


In [264]:
## Check
print( unzipbits == bits)

True


In [265]:
print(  len(zipbits) / len(unzipbits) * 100, "% in size" )
print("Encoded in %d bits compared to entropy %0.3f bits " %(len(zipbits), entropy))

48.2421875 % in size
Encoded in 247 bits compared to entropy 240.126 bits 
